# __Gesamtdatenanalyse__

In diesem Teil werden die Stelleninserate untersucht, indem die Wörter aus dem Wörterbuch im Inserat gezählt werden.

In [2]:
# Import der benötigten Libraries
import pandas as pd
import nltk
import numpy as np
import re
import os

pd.set_option('display.max_colwidth', 50)

# Working-Directory
os.chdir('C:/Users/Sven Konermann/Documents/Master/2_Semester/aF_E_Faelle/Unterlagen/00_all')
print(os.getcwd())

# Files im Directory
print(os.listdir())

C:\Users\Sven Konermann\Documents\Master\2_Semester\aF_E_Faelle\Unterlagen\00_all
['Anzahl_Inserate_Anforderungsdaten.pdf', 'Anzahl_Inserate_Gesamtdaten.pdf', 'Inserate_Anforderungsanalyse.csv', 'Inserate_clean_Anforderungsanalyse.csv', 'Inserate_clean_Gesamtdatenanalyse.csv', 'Inserate_clean_Test.csv', 'Inserate_Gesamtdatenanalyse.csv', 'Inserate_Gesamtdatenanalyse_.csv', 'my_fig.pdf', 'ngram_count_all_Data.csv', 'Subset_Accounting.csv', 'Subset_all.csv', 'Subset_ApplManager.csv', 'Subset_Business_Analyst.csv', 'Subset_Data_Science.csv', 'Subset_HR.csv', 'Subset_IT_Consulting.csv', 'Subset_IT_Projektmanagement.csv']


In [3]:
data = pd.read_csv("Inserate_clean_Gesamtdatenanalyse.csv", sep=";")
data = data[~data['Text_Anforderungen'].isnull()] # Wenn NULL-Werte, lösche diese
print(data.shape)

(40821, 10)


Suche nach Teil-Wörtern und Phrasen

In [6]:
# Einlesen vordefiniertes Wörterbuch
set_phrases = pd.read_excel('Phrasenwoerter.xlsx')
set_phrases.head()

,word,kategorie,kategorie_name,hauptkategorie
0,visual studio,2,Informatik Software-Kenntnisse,IT-Kenntnisse
1,big data,3,Datenmanagement,IT-Kenntnisse
2,business intelligence,3,Datenmanagement,IT-Kenntnisse
3,data science,3,Datenmanagement,IT-Kenntnisse
4,datenbank,3,Datenmanagement,IT-Kenntnisse


In [7]:
# Vergleichen unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()
count = 0
while year <= data['year'].max():
    wordsets_phrases = data.loc[data['year'] == year]['Text_Anforderungen'] # Text Detok, da n-grams nach dem gesucht wurden
    for word in set_phrases['word']:
        count = sum(1 for s in wordsets_phrases if word in s)
        index = set_phrases[set_phrases['word']==word].index
        set_phrases.loc[index, year] = count
        set_phrases.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))    
    
    year += 1

Suche nach Fix-Wörtern, die genau so vorkommen müssen

In [8]:
set_fix = pd.read_excel("Fixwoerter.xlsx")

In [9]:
# Anforderungen Tokenisieren
set_fix['text_tok'] = set_fix.apply(lambda row: nltk.word_tokenize(row['word']), axis=1)
set_fix['text_tok'].head(10)

0      [abap]
1         [c]
2      [java]
3     [plsql]
4    [python]
5         [r]
6    [apache]
7     [azure]
8    [citrix]
9      [hana]
Name: text_tok, dtype: object

In [10]:
# Anforderungen in einem Set speichern
new_set = set()
set_fix['text_tok'].apply(new_set.update)
u = new_set

In [11]:
# Vergleiche unique words, ob sie vorkommen im Text_new. Unique words list kommt vom Tokanizing oben.
year = data['year'].min()

while year <= data['year'].max():    
    wordsets_fix = [ frozenset(document.split(' ')) for document in data.loc[data['year'] == year]['Text_Anforderungen'] ]
    for word in u:
        count = sum( 1 for s in wordsets_fix if word in s )
        index = set_fix[set_fix['word']==word].index
        set_fix.loc[index, year] = count
        set_fix.loc[index, str(year) + ' %'] = (count / len(wordsets_phrases))

    year += 1

In [12]:
anforderungen = set_phrases.append(set_fix, ignore_index=True)

C:\Users\Sven Konermann\AppData\Local\Temp\ipykernel_26100\2611590460.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  anforderungen = set_phrases.append(set_fix, ignore_index=True)


In [13]:
# Anforderungen mit den meisten Nennungen
anforderungen.sort_values(by=2021,ascending=False).head(5)

,word,kategorie,kategorie_name,hauptkategorie,2014,2014 %,2015,2015 %,2016,2016 %,...,2017 %,2018,2018 %,2019,2019 %,2020,2020 %,2021,2021 %,text_tok
10,deutsch,8,Sprachkenntnisse,Sprachen,3796.0,0.650000,3346.0,0.661788,3645.0,0.698142,...,0.720772,4200.0,0.710780,3754.0,0.723593,3173.0,0.742222,2478.0,0.691406,NaN
11,englisch,8,Sprachkenntnisse,Sprachen,3270.0,0.559932,2747.0,0.543315,2619.0,0.501628,...,0.544015,3118.0,0.527670,2880.0,0.555127,2436.0,0.569825,1988.0,0.554688,NaN
21,teamfähig,12,Soft Skills,Soft Skills,1752.0,0.300000,1543.0,0.305182,1518.0,0.290749,...,0.280097,1786.0,0.302251,1594.0,0.307247,1370.0,0.320468,1269.0,0.354074,NaN
68,berufserfahrung,9,Berufserfahrung,Abschlüsse und Berufserfahrung,1614.0,0.450335,1397.0,0.389788,1344.0,0.375000,...,0.432199,1613.0,0.450056,1498.0,0.417969,1284.0,0.358259,1051.0,0.293248,[berufserfahrung]
66,sap,7,Nicht-Informatik Software-Kenntnisse,IT-Kenntnisse,1659.0,0.462891,1382.0,0.385603,1516.0,0.422991,...,0.440848,1519.0,0.423828,1187.0,0.331194,857.0,0.239118,785.0,0.219029,[sap]


In [14]:
# Nur für Daten mit Jobcloud
anforderungen.to_csv("ausgewertete_Anforderungen.csv", sep=";")